In [9]:
import pandas as pd
import re

original_covid_df = pd.read_csv("indicadores-covid.csv")

original_covid_df

null_count = original_covid_df.isnull().sum()
not_null_count = original_covid_df.notnull().sum()

total_count = len(original_covid_df)

In [127]:
nullItens = pd.DataFrame({
    "Coluna": original_covid_df.columns,
    "Nulos": null_count,
    "Preenchidos": not_null_count,
    "Total": total_count,
}).reset_index(drop=True)

nullItens

,Coluna,Nulos,Preenchidos,Total
0,source_id,0,10499,10499
1,dataNotificacao,9,10490,10499
2,dataInicioSintomas,757,9742,10499
3,sintomas,32,10467,10499
4,profissionalSaude,41,10458,10499
5,racaCor,631,9868,10499
6,outrosSintomas,6281,4218,10499
7,sexo,42,10457,10499
8,uf,5084,5415,10499
9,municipio,0,10499,10499


In [129]:
null_percentage = (null_count / total_count) * 100
not_null_percentage = (not_null_count / total_count) * 100

filling_df = pd.DataFrame({
    "Coluna": original_covid_df.columns,
    "Nulos": null_count,
    "Preenchidos": not_null_count,
    "Percent Nulos": null_percentage.round(2).astype(str).str.replace('.', ','),
    "Percent Preenchidos": not_null_percentage.round(2).astype(str).str.replace('.', ','),
    "Total": total_count,
}).reset_index(drop=True)

filling_df

,Coluna,Nulos,Preenchidos,Percent Nulos,Percent Preenchidos,Total
0,source_id,0,10499,"0,0","100,0",10499
1,dataNotificacao,9,10490,"0,09","99,91",10499
2,dataInicioSintomas,757,9742,"7,21","92,79",10499
3,sintomas,32,10467,"0,3","99,7",10499
4,profissionalSaude,41,10458,"0,39","99,61",10499
5,racaCor,631,9868,"6,01","93,99",10499
6,outrosSintomas,6281,4218,"59,82","40,18",10499
7,sexo,42,10457,"0,4","99,6",10499
8,uf,5084,5415,"48,42","51,58",10499
9,municipio,0,10499,"0,0","100,0",10499


In [11]:
# Validador para texto
def is_valid_text(value):
    if pd.isnull(value):
        return False
    return bool(re.fullmatch(r"[A-Z\s]+", value))

# Validador para data
def is_valid_date(value):
    if pd.isnull(value):
        return False
    return bool(re.fullmatch(r"\d{4}-\d{2}-\d{2}", value))

# Validador para idade
def is_valid_age(value):
    if pd.isnull(value):
        return False
    if isinstance(value, (int, float)):
        return value >= 0
    return False

# Validador para código de laboratório
def is_valid_lab_code(value):
    if pd.isnull(value):
        return False
    # Verifica se o valor contém apenas letras maiúsculas, espaços ou '/'
    return bool(re.fullmatch(r"[A-Z/\s]+", value))

# Função para padronizar código de laboratório
def standardize_lab_code(value):
    if pd.isnull(value):
        return value
    # Remove caracteres inválidos, mantém apenas letras, espaços e '/'
    value = re.sub(r"[^A-Za-z/\s]", "", value)
    # Converte para letras maiúsculas
    return value.upper()

# Dicionário para mapear colunas aos validadores
validators = {
    "text": {
        "columns": [
            "sintomas",
            "profissionalSaude",
            "racaCor",
            "outrosSintomas",
            "sexo",
            "uf",
            "municipio",
        ],
        "validator": is_valid_text,
    },
    "date": {
        "columns": [
            "dataNotificacao",
            "dataInicioSintomas",
            "dataPrimeiraDose",
            "dataSegundaDose",
        ],
        "validator": is_valid_date,
    },
    "age": {
        "columns": ["idade"],
        "validator": is_valid_age,
    },
    "lab_code": {
        "columns": [
            "codigoLaboratorioPrimeiraDose",
            "codigoLaboratorioSegundaDose",
        ],
        "validator": is_valid_lab_code,
    },
}

# Função para analisar e validar colunas
def analyze_column(df, column, validator):
    """Aplica o validador e calcula estatísticas de validação para uma coluna."""
    total_count = len(df)
    df[f"{column}_is_valid"] = df[column].apply(validator)
    valid_count = df[f"{column}_is_valid"].sum()
    invalid_count = total_count - valid_count
    return {
        "Coluna": column,
        "Válidos": f"{valid_count}",
        "Inválidos": f"{invalid_count}",
        "Percent Válidos": f"{(valid_count / total_count) * 100:.2f}".replace('.', ','),
        "Percent Inválidos": f"{(invalid_count / total_count) * 100:.2f}".replace('.', ','),
        "Total": total_count,
    }

# Aplicar padronização nos códigos de laboratório
for column in ["codigoLaboratorioPrimeiraDose", "codigoLaboratorioSegundaDose"]:
    if column in original_covid_df.columns:
        original_covid_df[column] = original_covid_df[column].apply(standardize_lab_code)

# Filtra as colunas existentes e realiza a análise
analysis_results = []
for key, config in validators.items():
    valid_columns = [col for col in config["columns"] if col in original_covid_df.columns]
    for column in valid_columns:
        result = analyze_column(original_covid_df, column, config["validator"])
        analysis_results.append(result)

# Gera o DataFrame com os resultados
standardization_df = pd.DataFrame(analysis_results)

# Exibe o DataFrame com os resultados de validação
standardization_df


,Coluna,Válidos,Inválidos,Percent Válidos,Percent Inválidos,Total
0,sintomas,0,10499,"0,00","100,00",10499
1,profissionalSaude,0,10499,"0,00","100,00",10499
2,racaCor,0,10499,"0,00","100,00",10499
3,outrosSintomas,1683,8816,"16,03","83,97",10499
4,sexo,171,10328,"1,63","98,37",10499
5,uf,5415,5084,"51,58","48,42",10499
6,municipio,0,10499,"0,00","100,00",10499
7,dataNotificacao,10490,9,"99,91","0,09",10499
8,dataInicioSintomas,9742,757,"92,79","7,21",10499
9,dataPrimeiraDose,5250,5249,"50,00","50,00",10499


In [12]:
from datetime import datetime

# Funções auxiliares
def is_date_in_range(value, start_date, end_date):
    """Verifica se a data está no intervalo especificado."""
    if pd.isnull(value):
        return True
    try:
        value_date = datetime.strptime(value, "%Y-%m-%d")
        return start_date <= value_date <= end_date
    except ValueError:
        return False


def is_first_date_before_second(first, second):
    """Verifica se a primeira data é anterior à segunda."""
    if pd.isnull(first) or pd.isnull(second):
        return True
    try:
        first_date = datetime.strptime(first, "%Y-%m-%d")
        second_date = datetime.strptime(second, "%Y-%m-%d")
        return first_date < second_date
    except ValueError:
        return False


# Mapeamento de regras e validadores
validators = {
    "dataNotificacao": {
        "range": ("2020-01-04", "2022-07-22"),
    },
    "dataInicioSintomas": {
        "range": ("2020-01-04", "2022-07-22"),
    },
    "dataPrimeiraDose": {
        "range": ("2020-03-23", "2022-07-22"),
        "relation": ("dataSegundaDose", "before"),
    },
    "dataSegundaDose": {
        "range": ("2020-08-19", "2022-07-22"),
        "relation": ("dataPrimeiraDose", "after"),
    },
    "codigoLaboratorioPrimeiraDose": {
        "valid_values": {"ASTRAZENECA/FIOCRUZ", "JANSSEN", "SINOVAC/BUTANTAN", "PFIZER"},
    },
    "codigoLaboratorioSegundaDose": {
        "valid_values": {"ASTRAZENECA/FIOCRUZ", "JANSSEN", "SINOVAC/BUTANTAN", "PFIZER"},
    },
    "profissionalSaude": {
        "valid_values": {"SIM", "NAO"},
    },
    "racaCor": {
        "valid_values": {"AMARELA", "BRANCA", "IGNORADO", "INDIGENA", "PARDA", "PRETA"},
    },
    "sexo": {
        "valid_values": {"MASCULINO", "FEMININO", "INDEFINIDO"},
    },
    "uf": {
        "valid_values": {
            "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG",
            "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO",
        },
    },
    "idade": {
        "valid_values": lambda x: pd.notnull(x) and isinstance(x, (int, float)) and x < 122,
    },
}


def validate_column(df, column, config):
    """Valida uma coluna com base nas regras especificadas no config."""
    results = {}

    if "range" in config:
        start_date, end_date = map(lambda x: datetime.strptime(x, "%Y-%m-%d"), config["range"])
        df[f"{column}_range_valid"] = df[column].apply(lambda x: is_date_in_range(x, start_date, end_date))
        results["range"] = df[f"{column}_range_valid"].sum()

    if "relation" in config:
        related_column, relation_type = config["relation"]
        if relation_type == "before":
            df[f"{column}_relation_valid"] = df.apply(
                lambda row: is_first_date_before_second(row[column], row[related_column]), axis=1
            )
        elif relation_type == "after":
            df[f"{column}_relation_valid"] = df.apply(
                lambda row: is_first_date_before_second(row[related_column], row[column]), axis=1
            )
        results["relation"] = df[f"{column}_relation_valid"].sum()

    if "valid_values" in config:
        valid_values = config["valid_values"]
        if callable(valid_values):
            df[f"{column}_valid_values"] = df[column].apply(valid_values)
        else:
            df[f"{column}_valid_values"] = df[column].apply(lambda x: x in valid_values if pd.notnull(x) else False)
        results["valid_values"] = df[f"{column}_valid_values"].sum()

    return results


# Processamento e coleta de resultados
consistency_results = []

for column, config in validators.items():
    if column not in original_covid_df.columns:
        continue
    column_results = validate_column(original_covid_df, column, config)
    for rule, valid_count in column_results.items():
        consistency_results.append({
            "Coluna": column,
            "Regra": rule,
            "Consistentes": valid_count,
            "Inconsistentes": total_count - valid_count,
            "Total": total_count,
        })

# Gera o DataFrame final de resultados
consistency_df = pd.DataFrame(consistency_results)

# Exibe os resultados
consistency_df


,Coluna,Regra,Consistentes,Inconsistentes,Total
0,dataNotificacao,range,10490,9,10499
1,dataInicioSintomas,range,10457,42,10499
2,dataPrimeiraDose,range,10477,22,10499
3,dataPrimeiraDose,relation,10485,14,10499
4,dataSegundaDose,range,10499,0,10499
5,dataSegundaDose,relation,10485,14,10499
6,codigoLaboratorioPrimeiraDose,valid_values,5204,5295,10499
7,codigoLaboratorioSegundaDose,valid_values,4348,6151,10499
8,profissionalSaude,valid_values,0,10499,10499
9,racaCor,valid_values,0,10499,10499


In [117]:
import pandas as pd

def analyze_uniqueness(df, column):
    # Identificar registros duplicados
    is_duplicated = df[column].duplicated(keep=False)

    # Contar registros duplicados e não duplicados
    duplicated_count = is_duplicated.sum()
    not_duplicated_count = len(df) - duplicated_count

    # Criar DataFrame com os resultados
    results_df = pd.DataFrame({
        "Duplicados": [duplicated_count],
        "Unicos": [not_duplicated_count],
        "Total": total_count,
    })

    return results_df

uniqueness_analysis_df = analyze_uniqueness(original_covid_df, "source_id")

uniqueness_analysis_df

,Duplicados,Unicos,Total
0,20,10479,10499


In [5]:
import pandas as pd


def calculate_uf_distribution(df, column, valid_ufs=None):
    """
    Calcula a distribuição de valores de UF em uma coluna específica de um DataFrame.
    
    Args:
        df (pd.DataFrame): O DataFrame de entrada.
        column (str): O nome da coluna contendo os valores de UF.
        valid_ufs (list, optional): Lista de UFs válidas. Caso não seja fornecida, será usada uma lista padrão.
    
    Returns:
        pd.DataFrame: DataFrame contendo a distribuição de UFs e suas porcentagens.
    """
    if valid_ufs is None:
        valid_ufs = [
            "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", 
            "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"
        ]

    # Contar ocorrências para cada UF válida
    uf_counts = df[column].value_counts()

    # Filtrar apenas UFs válidas e calcular as porcentagens
    total_count = len(df)
    results = [
        {
            "UF": uf,
            "Ocorrências": uf_counts.get(uf, 0),
            "Percentual": f"{(uf_counts.get(uf, 0) / total_count) * 100:.2f}".replace('.', ','),
            "Total": total_count,
        }
        for uf in valid_ufs
    ]

    return pd.DataFrame(results)


# Exemplo de uso
uf_distribution_df = calculate_uf_distribution(original_covid_df, "uf")

# Exibe os resultados
uf_distribution_df


,UF,Ocorrências,Percentual,Total
0,AC,0,"0,00",10499
1,AL,5,"0,05",10499
2,AP,2,"0,02",10499
3,AM,1,"0,01",10499
4,BA,14,"0,13",10499
5,CE,4,"0,04",10499
6,DF,8,"0,08",10499
7,ES,1,"0,01",10499
8,GO,30,"0,29",10499
9,MA,3,"0,03",10499
